In [ ]:
%matplotlib inline

In [ ]:
import numpy as np
import pylab as plt
import h5py
from glob import glob
import os
from skimage.filters import median
import cv2

In [ ]:
def log_progress(sequence, every=None, size=None, name='Items'):
    from ipywidgets import IntProgress, HTML, VBox
    from IPython.display import display

    is_iterator = False
    if size is None:
        try:
            size = len(sequence)
        except TypeError:
            is_iterator = True
    if size is not None:
        if every is None:
            if size <= 200:
                every = 1
            else:
                every = int(size / 200)     # every 0.5%
    else:
        assert every is not None, 'sequence is iterator, set every'

    if is_iterator:
        progress = IntProgress(min=0, max=1, value=1)
        progress.bar_style = 'info'
    else:
        progress = IntProgress(min=0, max=size, value=0)
    label = HTML()
    box = VBox(children=[label, progress])
    display(box)

    index = 0
    try:
        for index, record in enumerate(sequence, 1):
            if index == 1 or index % every == 0:
                if is_iterator:
                    label.value = '{name}: {index} / ?'.format(
                        name=name,
                        index=index
                    )
                else:
                    progress.value = index
                    label.value = u'{name}: {index} / {size}'.format(
                        name=name,
                        index=index,
                        size=size
                    )
            yield record
    except:
        progress.bar_style = 'danger'
        raise
    else:
        progress.bar_style = 'success'
        progress.value = index
        label.value = "{name}: {index}".format(
            name=name,
            index=str(index or '?')
        )

In [ ]:
ufo_dir = '/diskmnt/a/makov/tomo_data/ufo/test/raw/'
files = glob(os.path.join(ufo_dir, '*.raw'))

In [ ]:
def read_file(file_path):
    frames = np.fromfile(file_path, dtype='<u2')
    x = frames.reshape(-1,3840, 5120)
    x = np.mean(x, axis=0).astype(np.float32)
    x = cv2.medianBlur(x,3)
    return x

with h5py.File('frames.h5') as h5f:
    for frame_file in log_progress(files):
        try:
            x = read_file(frame_file)
            name = os.path.split(frame_file)[-1].split('.')[0]
            h5f[name] = x
        except Exception as e:
            print(e)

In [ ]:
with h5py.File('frames.h5') as h5f:
    empty_keys = [k for k in h5f if k.endswith('_e') if not k == 'mult2_e']
    empty = None
    
    for ek in empty_keys:
        t = h5f[ek].value
        if empty is None:
            empty = t
        else:
            empty += t
            
#         plt.figure(figsize=(10,10))
#         plt.imshow(t)
#         plt.title(ek)
#         plt.colorbar(orientation = 'horizontal')
#         plt.show()
    
    empty /= len(empty_keys)
    
    dark_keys = [k for k in h5f if k.endswith('_d')]
    
    dark = None
    for dk in dark_keys:
        t = h5f[dk].value
        if dark is None:
            dark = t
        else:
            dark += t
            
#         plt.figure(figsize=(10,10))
#         plt.imshow(t)
#         plt.title(dk)
#         plt.colorbar(orientation = 'horizontal')
#         plt.show()
    
    dark /= len(dark_keys)


In [ ]:
with h5py.File('frames.h5') as h5f:
    data_keys = [k for k in h5f if not k.endswith('_e') if not k.endswith('_d')]
#     print(list(data_keys))
    
    data_frames = {}
    for dk in data_keys:
        data_frames[int(dk[4:])] = h5f[dk].value

In [ ]:
for k, v in data_frames.items():
    plt.imsave('{:003}.png'.format(k),
               np.rot90((data_frames[k]-dark)/(empty-dark), 3)[1000:4500,500:3000],
               vmin=0, vmax=1, cmap='gray')

In [ ]:
plt.figure(figsize=(10,10))
plt.imshow(empty)
plt.title('empty')
plt.colorbar(orientation = 'horizontal')


In [ ]:
plt.figure(figsize=(10,10))
plt.imshow(dark)
plt.title('dark')
plt.colorbar(orientation = 'horizontal')


In [ ]:
plt.figure(figsize=(10,10))
plt.imshow(empty-dark)
plt.title('empty-dark')
plt.colorbar(orientation = 'horizontal')
